## Machine Learning project using Python

 O objetivo dessa lista de exercícios é desenvolver um projeto de Machine Learning usando uma base já conhecida na literatura. O título do exercício deve ser "Um exercício de Machine Learning usando a base XXX do Kaggle." Espera-se que cada base seja usada por no máximo um estudante. Logo, se você decidir fazer um desses exercícios, explicite na planilha que base você pretende estudar.

Considere que você está enfrentando um projeto real de machine learning. Então, sugere-se que, para resolver esse exercício, você considere os seguintes passos abaixo:

**1)** Defina o seu problema (objetivo): O que você está tentando prever? Que tipo de problema você tem? Supervisionado? Não-supervisionado?

**2)** Colete os dados: Quais são as variáveis disponíveis? Estatísticas descritivas? As variáveis são contínuas? Categóricas? Binárias? Possuem variáveis multicolineares? Existem dados faltantes (*missing data*)? Como lidar com missing data? Você precisará enriquecer sua base?  Você precisará normalizar a sua base de dados? Histogramas de variáveis (suas variáveis possuem dinâmica)? Correlação entre variáveis?

**3)** Divida os dados em treino/teste: Sua base é balanceada? Você precisa pensar em alguma estratégia especial para dividir a sua base? Você considera estratégias como *K-fold validation*?

**4)** Construa o seu *benchmark*. Sempre comece com modelo bem simples que você tenha controle. Uma boa escolha é um modelo linear em que  você consiga interpretar os seus resultados/ter uma ideia das variáveis que podem influenciar os seus resultados.

**5)** Você precisará de um procedimento específico para escolha dos hiperparâmetros?

**6)** Você precisará de um procedimento específico de seleção de variáveis?

**7)** Você precisará regularizar o seu modelo? Como você pretende regularizá-los?

**8)** Modelos industriais são suficientes (modelos de regressão, árvores de decisão, *SVM*)?

**9)** Você precisará trabalhar com redes neurais rasas ou profundas?

Para ajudar a sua solução, você pode buscar ideias em soluções prévias disponíveis na literatura/internet, mas o código deve ser desenvolvido por você. Finalizando, o conteúdo usado para solucionar essas questões é todo o material do curso e não dessa aula. Para resolver essa questão, você pode usar qualquer base de dados disponível em: https://www.kaggle.com/datasets

****

# 1) Define your problem.

In this project, I shall predict some movies box office. In order to do so, I'll use [The Movie Database's (TMDB) dataset](https://www.kaggle.com/c/tmdb-box-office-prediction/overviews), containing information like the movies' cast, release data, their original language, runtime and genra. Since I already have the revenue for the train dataset, it will be a *supervised* problem; therefore, some possible algorithms are linear regression models, support vector regressions (SVR), decision tree regressions and random forest regressions.

# 2) Collect the data

### Importing the dataset

In [ ]:
# DataFrames and Arrays
import pandas as pd
import numpy as np

# Plots
import matplotlib.pyplot as plt
import seaborn as sns

# Other
import zipfile # To deal with zips
import ast # Parsing dictionary variables
import requests # Web scraping
import time # For the requests' sleep
from collections import Counter # Counts occurrences in dictionaries
import cpi # Inflation adjustments
from datetime import date

# Guarantees code will accurately inflate to today's dollar
cpi.update()

# Random seed
np.random.seed(0)

In [ ]:
# The datasets
zf = zipfile.ZipFile('datasets/tmdb-box-office-prediction.zip') 
train = pd.read_csv(zf.open('train.csv'))
test = pd.read_csv(zf.open('test.csv'))

In [ ]:
# Columns that are JSON objects
dict_cols = ['genres', 'production_companies', 'production_countries', 'cast', 'crew']

def text_to_dict(df):
    """ Transforms JSON columns from strings to dictionaries """
    for col in dict_cols:
        df[col] = df[col].apply(lambda x: {} if pd.isna(x) else ast.literal_eval(x) )
    return df

for df in [train, test]:
    df = text_to_dict(df)

In [ ]:
print("Shape of the train dataset: ", train.shape)
print("Shape of the test dataset: ", test.shape)

Note that the test dataset is larger than the train set, which is unusual. Also, the test dataset does not contain any information on the target variable (the movies' revenue). Let's take a look first at the train dataset.

In [ ]:
# A brief look at the data
train.head() # First five observations

In [7]:
train.info() # Variable types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     3000 non-null   int64  
 1   belongs_to_collection  604 non-null    object 
 2   budget                 3000 non-null   int64  
 3   genres                 3000 non-null   object 
 4   homepage               946 non-null    object 
 5   imdb_id                3000 non-null   object 
 6   original_language      3000 non-null   object 
 7   original_title         3000 non-null   object 
 8   overview               2992 non-null   object 
 9   popularity             3000 non-null   float64
 10  poster_path            2999 non-null   object 
 11  production_companies   3000 non-null   object 
 12  production_countries   3000 non-null   object 
 13  release_date           3000 non-null   object 
 14  runtime                2998 non-null   float64
 15  spok

Note that *release_date* variable was compiled as an "object" type (ie, probably a *string* variable), instead of *datetime*. All other variables are assigned to their correct types. It will be corrected further on.


## Filling Test's revenues and merging it to Train dataset

In order to increase the number of observations, I will scrap the revenue values of the movies from 'test' dataset using the TMDB's Application Programming Interface (API) and, subsequently, merge both train and test datasets into a single "complete" dataset. The train-test split will then be done later on, using this complete dataset.

### TMDB's API

The TMDB's API is a free interface which only requires you to create an account in order to use it. Once you're all set, you can perfom "GET" requests using, for instance, the Python's "request" package. The request will return a JSON object, which is nothing more than a string object containing dictionaries of elements. The JSON object can then be parsed (interpreted) thereafter using requests' ".json()" method.

Unfortunately, the movies' TMDB ids are required to obtain the movies' revenues, and they are not directly available from our datasets. However, it is possible to return them perfoming requests using the TMDB API's "find" method.

In [8]:
def get_TMDB_id(imdb_id, api_key):
    """ Returns the tmdb_id for a given imdb_id, using TMDB API's "find" method
    
    imdb_id: self-explanatory
    api_key: your TMDB API key.
    """
    url = "https://api.themoviedb.org/3/find/" + imdb_id

    # The parameters to be used on the request
    querystring = {"api_key": api_key,
                   "language":"en-US",
                   "external_source":"imdb_id"}
    
    # Making the request and parsing its text from JSON format to Python's dictionaries
    response = requests.request("GET", url, params=querystring)
    parsed_response = response.json()
    
    # Getting the tmdb_id
    tmdb_id = parsed_response['movie_results'][0]['id']
    
    return str(tmdb_id)


def get_revenue(tmdb_id, api_key):
    """ Given the movie's tmdb_id, returns its revenue, using TMDB API's "movie" method
    
    tmdb_id: self-explanatory
    api_key: your TMDB API key
    """
    url = "https://api.themoviedb.org/3/movie/" + tmdb_id

    querystring = {"api_key": api_key}

    # Getting the request object
    response = requests.request("GET", url, params=querystring)

    # Parsing the information we want
    parsed_response = response.json()
    revenue = parsed_response['revenue']
    
    return revenue

Now, let's make the requests. It is worth noting that you will need your own API key in order to do so. Also, it is highly recommended to do a limited number of requests *per minute* to avoid overloading the system and, hence, being banned from it. I'll limit them to 20 *per minute*. 

Finally, note also that this procedure will take approximately 7 hours to be done.

In [9]:
# Creating lists to receive the imdb_ids and also the revenues
test_imdb_ids = [x for x in test['imdb_id']]
the_revenues = [0] * len(test_imdb_ids)
# api_key = "" # YOUR API_KEY HERE

In [10]:
"""
# WARNING: the following code chunk requires a TMDB API key (which you can get for free on their website)
# It also takes about 7 hours to run. If you __REALLY__ want to run it, replace "if False:" on the following
# line of code to "if True:"

if False:
    for i in range(len(test_imdb_ids)):
        # Fill the values
        print(i)
        try:
            next_tmdb_id = get_TMDB_id(test_imdb_ids[i], api_key)
            the_revenues[i] = get_revenue(next_tmdb_id, api_key)
        except:
            print("An error occurred on observation ", i)

        # Avoids excessive requests
        # After 20 requests, stops the execution for one minute
        # (remember: each loop counts as 2 requests)
        if (i + 1) % 10 == 0:
            print("Step: ", i)
            time.sleep(60)

    print("\nDone!")
"""

'\n# WARNING: the following code chunk requires a TMDB API key (which you can get for free on their website)\n# It also takes about 7 hours to run. If you __REALLY__ want to run it, replace "if False:" on the following\n# line of code to "if True:"\n\nif False:\n    for i in range(len(test_imdb_ids)):\n        # Fill the values\n        print(i)\n        try:\n            next_tmdb_id = get_TMDB_id(test_imdb_ids[i], api_key)\n            the_revenues[i] = get_revenue(next_tmdb_id, api_key)\n        except:\n            print("An error occurred on observation ", i)\n\n        # Avoids excessive requests\n        # After 20 requests, stops the execution for one minute\n        # (remember: each loop counts as 2 requests)\n        if (i + 1) % 10 == 0:\n            print("Step: ", i)\n            time.sleep(60)\n\n    print("\nDone!")\n'

Saving the revenue values to a file

In [11]:
#pd.DataFrame(the_revenues).to_excel('test_revenues.xlsx', header=True, index=False)

Loading the file

In [12]:
test['revenue'] = pd.read_excel('datasets/test_revenues.xlsx')

Dropping the movies for which the information on the revenue was not available, and merging the dataframes thereafter.

In [13]:
test = test.drop(test[test.revenue == 0].index) # drops those movies for which the revenue was not available

# Merging the dataframes into a single df.
frames = [train, test]
complete_df = pd.concat(frames)

# Removes missing data from "budget" variable
complete_df = complete_df.loc[complete_df['budget'] != 0]

Although there is a total of 7345 observations in the complete dataframe, about 2000 of them have budget value equal to zero, which can be interpreted as missing data. Indeed, I have tried both inputting the mean/median value and also dropping this missing data, and the latter perfomed way better in all metrics on cross-validation. Thus, I opted to drop this missing data.

## Data Prep

#### Date variables

It is worth noting that the competition was supposed to conclude May 30, 2019. Therefore, all movies with year greater than 19 are from the last century, consequently.

In [14]:
def gen_year(x):
    """ Returns the year from the date.
    
    PS: the release date was originally a STRING on the "mm/dd/yy" format
    """
    year = x.split('/')[2]
    year = int(year)
    return year

##### Year

Creating the year variable

In [15]:
# Creating the YEAR variable
complete_df['year'] = 0
complete_df['year'] = complete_df['release_date'].apply(lambda x: gen_year(x))

In [16]:
# Counting the occurrences of years between 1920 and 1930
print("Absolute frequency of movies with release dates between 1920 and 1930")
complete_df.loc[(complete_df['year'] <= 30) & (complete_df['year'] >= 20)]['year'].value_counts()

Absolute frequency of movies with release dates between 1920 and 1930


25    5
28    4
27    2
26    2
30    1
29    1
24    1
22    1
21    1
Name: year, dtype: int64

As there are almost no movies in the dataset from 1930 or older, it is reasonable to label all movies with year ending in 19 or below as 21th century movies.

In [17]:
def fix_year_release_date(release_date):
    """ Adds 1900 or 2000 to the 'release_date' variable's year"""
    year = release_date.split('/')[2] # Picks the year
    
    # Corrects the year
    if int(year) <= 19:
        return release_date[:-2] + '20' + year
    else:
        return release_date[:-2] + '19' + year

In [18]:
# Corrects the 'year' column
complete_df['year'] = complete_df['year'].apply(lambda x: 1900 + x if x > 19 else 2000 + x)

##### Release date

In [19]:
# Corrects the 'release_date' column
complete_df['release_date'] = complete_df['release_date'].apply(lambda x: fix_year_release_date(x))
complete_df['release_date'] = pd.to_datetime(complete_df['release_date'])

##### Quarter and weekday

I tried also using release months instead of release quarters, but there was almost no performance improvements on cross-validation. Thus, I opted for the quarters.

In [20]:
complete_df['release_quarter'] = complete_df['release_date'].dt.quarter
complete_df['release_weekday'] = complete_df['release_date'].dt.weekday

In [21]:
# Monday == 0, Friday == 4, Sunday == 6
complete_df['release_weekday'].head()

0    4
1    4
2    4
3    4
5    3
Name: release_weekday, dtype: int64

In [22]:
complete_df['release_date'].head()

0   2015-02-20
1   2004-08-06
2   2014-10-10
3   2012-03-09
5   1987-08-06
Name: release_date, dtype: datetime64[ns]

#### Revenue and Budget

Adjusting the values for inflation and creating the natural log version of the variables:

In [ ]:
# Getting the most recent movie release date
max_date = complete_df['release_date'].max()

# Inflation adjusting
complete_df['revenue'] = complete_df.apply(lambda x: cpi.inflate(value = x.revenue, year_or_month = x.release_date, to = max_date), axis = 1)
complete_df['budget'] = complete_df.apply(lambda x: cpi.inflate(value = x.budget, year_or_month = x.release_date, to = max_date), axis = 1)

# log1p(x) = ln(x+1): it avoids calculating log(0), which is undefined
complete_df['ln_revenue'] = complete_df['revenue'].apply(lambda x: np.log1p(x))
complete_df['ln_budget'] = complete_df['budget'].apply(lambda x: np.log1p(x))

#### Popularity

As popularity has a skewed distribution, we shall create log popularity. I've also tested using (popularity, popularity^2, popularity^3 and popularity^4) against log popularity, and the latter perfomed better in all metrics.

In [ ]:
complete_df['ln_popularity'] = complete_df['popularity'].apply(lambda x: np.log1p(x))

#### Runtime variable

There are about 15 movies without their runtime information, or with 0 (zero) minutes runtime, which is almost the same, since it is virtuallyy impossible for a movie to be less than one minute long. However, all of this can be easily fixed by searching on IMDB.

In [ ]:
# Finding out the movies with NaN (missing data) on runtime variable
complete_df['runtime'] = complete_df['runtime'].replace(0.0, np.nan) # Replacing 0 with NaN
complete_df.loc[complete_df['runtime'] != complete_df['runtime'], ['id', 'title', 'runtime', 'imdb_id']]

In [ ]:
# Filling the runtime's missing values with IMDB's information
complete_df.loc[complete_df['id'] == 1336,'runtime'] = 130 # Korolyov
complete_df.loc[complete_df['id'] == 3244,'runtime'] = 93 # La caliente niña Julietta
complete_df.loc[complete_df['id'] == 4490,'runtime'] = 91 # Pancho, el perro millonario
complete_df.loc[complete_df['id'] == 4633,'runtime'] = 100 # Nunca en horas de clase
complete_df.loc[complete_df['id'] == 6818,'runtime'] = 90 # Miesten välisiä keskusteluja
complete_df.loc[complete_df['id'] == 391,'runtime'] = 96 # The Worst Christmas of My Life
complete_df.loc[complete_df['id'] == 978,'runtime'] = 93 # La peggior settimana della mia vita
complete_df.loc[complete_df['id'] == 1542,'runtime'] = 93 # All at Once
complete_df.loc[complete_df['id'] == 2151,'runtime'] = 108 # Mechenosets
complete_df.loc[complete_df['id'] == 2499,'runtime'] = 86 # Hooked on the Game 2. The Next Level
complete_df.loc[complete_df['id'] == 2866,'runtime'] = 96 # Tutto tutto niente niente
complete_df.loc[complete_df['id'] == 4074,'runtime'] = 103 # Shikshanachya Aaicha Gho
complete_df.loc[complete_df['id'] == 4431,'runtime'] = 96 # Plus one
complete_df.loc[complete_df['id'] == 5520,'runtime'] = 86 # Glukhar v kino
complete_df.loc[complete_df['id'] == 5849,'runtime'] = 140 # Shabd
complete_df.loc[complete_df['id'] == 6210,'runtime'] = 104 # The Last Breath

In [ ]:
# Creating a "budget/runtime" ratio variable
complete_df['budget_runtime_ratio'] = complete_df.budget / complete_df.runtime

Another variable I created was "budget to runtime ratio". I tested also the implementation of a "squared runtime" variable, but it resulted in little performance improvement to the models.

#### Crew and cast

For the crew and cast, we'll create a variable for each with their size, and also a dummy variable "top_50", which takes 1 if the movie contains at least one of the top 50 most recurrent cast/crew member in the dataset, and 0 elsewise.

In [ ]:
class json_variables(object):
    """ Handles JSON (ie, dictionary) variables. """
    
    def __init__(self, df, variable, top_number):
        """ Initiates the class.
        
        df: dataframe.
        variable: the variable of interest (cast, crew, prod.companies, prod. countries or genre.)
        top_number: threshold. Example: top_number = 30 means it will consider only the 30
        most frequent cast / crew members / etc in the dataframe
        """
        self.df = df
        self.variable = variable
        self.top_number = top_number
        
        # Creates a list with each observation from that variable
        self._list_of_obs = list(df[variable].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
        
        # Counts the number of occurrences for the top "top_number" cast / crew members on the df,
        # (dictionary-like list, with tuples containing the names followed by their counter)
        self.top_variable = Counter([i for j in self._list_of_obs for i in j]).most_common(top_number)
        
        # Grab only the cast / crew names, without their occurrences counter
        self.top_variable_names = [x[0] for x in self.top_variable]

        
        return None

    
    def method(self, select):
        """ Selects whether to call "generate_counter_var" or "generate_dummies"
        
        select: selected method name (counter, dummy)
        """
        if (select != "counter") and (select != "dummy"):
            raise ValueError("Error. Selection variable must be either 'counter' or 'dummy'")
        
        # Getting rid of "self"
        variable = self.variable
        top_number = self.top_number
        
        # Creates new df to add the brand new variable
        new_df = self.df
        
        # Creates a new string variable containing all the crew / cast members on df
        new_df[variable + '_all'] = new_df[variable].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
        
        
        # Selection
        if select == "counter":
            new_df = self.generate_counter_var(variable, top_number, new_df)
        else: # ie, if select == "dummy"
            new_df = self.generate_dummies(variable, top_number, new_df)
            

        # Removes support variables created
        new_df.drop([variable + '_all'], axis = 1, inplace = True)
        
        return new_df
    
    
    def generate_counter_var(self, variable, top_number, new_df):
        """ Adds a variable to the df counting how many "top_number" cast / crew members are there on each movie. 
        
        new_df: copy from the original df
        """
    
        def occurrence_counter(df_variable, list_of_names):
            """ Counts number of famous cast / crew members on each movie """
            occurrences = 0
            
            for person in list_of_names:
                if person in df_variable:
                    occurrences += 1
                    
            return occurrences
        
        # Applies the previously defined function
        new_df[variable + '_top_' + str(top_number) + '_counter'] = 0
        new_df[variable + '_top_' + str(top_number) + '_counter'] = new_df[variable + '_all'].apply(lambda x: occurrence_counter(x, self.top_variable_names))
        
        return new_df
    
    
    def generate_dummies(self, variable, top_number, new_df):
        """ Creates dummies taking account if the movie belongs to the "top_number"
        genra / or was developed by the "top_number" company
        """
        # Creates dummy variables
        for entry in self.top_variable_names:
            new_df[variable + '_' + entry] = complete_df[variable + '_all'].apply(lambda x: 1 if entry in x else 0)
            
        return new_df

In [ ]:
# Size
for variable in ['crew', 'cast']:
    complete_df[variable + '_size'] = complete_df[variable].apply(lambda x: len(x))

# Top_50 dummies
for variable in ['cast', 'crew']:
    my_object = json_variables(complete_df, variable, 50)
    complete_df = my_object.method(select = "counter")

#### Production Countries, production companies and genra

For all these three variables, we will create variables counting their occurrences. For production companies, we'll also create dummy variables for the top 10 most common occurrences. For production countries, we'll do something different, however.

In [ ]:
# Renaming "production" to "prod", for the sake of simplicity
complete_df.rename(columns={"production_countries": "prod_countries", "production_companies": "prod_companies"}, inplace = True)

# Counter
for variable in ['prod_countries', 'prod_companies', 'genres']:
    complete_df[variable + '_count'] = complete_df[variable].apply(lambda x : len(x))

In [ ]:
# Production companies and genres dummies
for variable in ['prod_companies', 'genres']:
    if variable == 'prod_companies':
        x = 11
    else:
        x = 10
    my_object = json_variables(complete_df, variable, x)
    complete_df = my_object.method(select = "dummy")

Since all movies from "Columbia Pictures Corporation" are also counted as "Columbia Pictures" movies, we'll drop the first.

In [ ]:
# Since all movies from "Columbia Pictures Corporation" are also counted as "Columbia Pictures" movies, we'll drop the first.
complete_df.drop(['prod_companies_Columbia Pictures Corporation'], axis = 1, inplace = True)

Since more than half of the movies was produced in the United States, we'll create a dummy for it. Also, we'll create a dummy for being produced on at least one of the other top 9 most common countries. Note that a movie may be produced in more than one country.

In [ ]:
my_object = json_variables(complete_df, 'prod_countries', 10)
top_10_countries = my_object.top_variable_names

# United States dummy
complete_df['prod_countries_all'] = complete_df['prod_countries'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
complete_df['prod_countries_USA'] = complete_df['prod_countries_all'].apply(lambda x: 1 if 'United States of America' in x else 0)


def produced_on_other_top_10(x, top_10_countries):
    """ Returns 1 if the movie was produced in at least one of the other top 10
    most common countries, and 0 otherwise
    """
    # Excludes USA from the list
    other_top_10_countries = top_10_countries[1:]
    
    for country in other_top_10_countries:
        if country in x:
            return 1
        else:
            pass
    
    return 0


# Other top 10 countries dummy:
complete_df['prod_countries_other_top_10'] = complete_df['prod_countries_all'].apply(lambda x: produced_on_other_top_10(x, top_10_countries))

#### Homepage variable

It is worth noting that about two-thirds of the movies have no homepage. Perhaps,this may be itself an useful information. Let's check it.

In [ ]:
# Creates a variable for having a homepage
complete_df['has_homepage'] = 0
complete_df.loc[complete_df['homepage'].isnull() == False, 'has_homepage'] = 1

In [ ]:
complete_df['has_homepage'].value_counts() # Counts the occurrences

#### Belongs to collection

As with the homepage information, I created a dummy for whether the movie belongs to a collection or not.

In [ ]:
complete_df['has_collection'] = complete_df['belongs_to_collection'].apply(lambda x: int(0) if x != x else int(1))

#### Original Language and spoken languages

For the sake of simplicity, since about 86% of the movies are in english, we'll create a dummy "original_lang_en" which takes 1 if the original language is english, and 0 elsewise. Also, we'll discard the "spoken languages" variable, as it probably is strongly correlated to the previous variable.

In [ ]:
complete_df['original_lang_en'] = complete_df['original_language'].apply(lambda x: int(1) if x == "en" else int(0))

#### Status variable

Last, but not least, for some reason, there are four movies labeled as "rumored" or in "post production" (*ie*, not yet released). 

In spite of this, all of them have revenues values. Furthermore, when checking on the internet, one will find that all of them have been released, indeed. Hence, as only a minimal fraction of the movies are labeled as something other than "released", it is reasonably safe to drop this feature without loss of generality.

In [ ]:
complete_df['status'].value_counts()

In [ ]:
complete_df[complete_df['status'] != 'Released'][['title','status','release_date','revenue']]

## Exploratory Data Analysis (EDA)

#### Year

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
sns.histplot(data = complete_df, x = "year")
plt.subplot(1, 2, 2)
sns.boxplot(x="year", y="ln_revenue", data=complete_df)
plt.xticks(rotation=90)
sns.despine() # Removes some of the borders

It is visible the increase in the number of movies along the years. The apparent decrease in 2018 is due to the lack of movies in the dataset after 2018-08-01. Also, the movies' revenue shows increasing volatily along the years.

#### Release weekday

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
sns.histplot(data = complete_df, x = "release_weekday").set(xlabel = "Release weekday (0 == Monday)", ylabel = "Frequency")

# Second Plot
plt.subplot(1, 2, 2)
sns.boxplot(x="release_weekday", y="ln_revenue", data=complete_df)
sns.despine() # Removes some of the borders
plt.show()

It is visible that the majority of movies has been released on a thursday or a friday. Furthermore, movies released on wednesday apparently have higher revenues.

Most of the movies are released on fridays. Also, the least profitable release days seems to be sunday, monday and saturday

#### Release quarter

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
sns.histplot(data = complete_df, x = "release_quarter").set(xlabel= "Release quarter", ylabel = "Frequency")

plt.subplot(1, 2, 2)
sns.boxplot(x="release_quarter", y="ln_revenue", data=complete_df).set(xlabel= "Release quarter", ylabel = "Log-revenue")
sns.despine()

The most profitable quarter seems to be the last one, which is intuitive. However, surprisingly, the quarter with the most releases in the dataset is the third. Indeed, september was the month with the most releases.

#### Budget

Log-budget *versus* log-revenue.

In [ ]:
ax = sns.regplot(x="ln_budget", y="ln_revenue", data=complete_df)
ax.set_title("Log-budget versus log-revenue", fontsize = 15)
ax.set_ylabel('Log-revenue')
ax.set_xlabel('Log-budget')
plt.show()

The higher the budget, the higher the revenue, apparently.

#### Popularity

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
sns.histplot(data = complete_df, x='popularity')
plt.ylabel("Absolute frequency")
plt.xlabel("Popularity")

plt.subplot(1, 2, 2)
sns.histplot(data = complete_df, x='ln_popularity')
plt.xlabel("Log-popularity")
plt.ylabel("Absolute frequency")
plt.show()

#### Runtime variable

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
sns.set_theme(style="whitegrid")
sns.histplot(data = complete_df, x = "runtime").set(title='Runtime variable histogram', xlabel='Runtime', ylabel='Count')
sns.despine() # Removes some of the borders

plt.subplot(1, 2, 2)
sns.scatterplot(data = complete_df, x='runtime', y='ln_revenue')
plt.title("Runtime vs. revenue")
plt.ylabel("Log-revenue")
plt.xlabel("Runtime")
plt.show()

In [ ]:
ax = sns.scatterplot(x="budget_runtime_ratio", y="ln_revenue", data=complete_df)
ax.set_title("Budget to runtime ratio vs. log-revenue", fontsize = 15)
ax.set_ylabel('Revenue')
ax.set_xlabel('Budget to runtime ratio')
plt.show()

Most of the movies are about 100 minutes long.

#### Original language

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
sns.histplot(data = complete_df.loc[complete_df['original_language'].isin(complete_df['original_language'].value_counts().head(10).index)], x = "original_language", bins = 5)
plt.title('Language count')
plt.xlabel("Original languages")

# Second plot
plt.subplot(1, 2, 2)
sns.boxplot(x='original_lang_en', y='ln_revenue', data=complete_df)
plt.title('Revenue per language')
plt.ylabel("Log-revenue")
plt.xlabel("Is english the movie's original language?")
sns.despine()

Movies in english reach higher revenues, and english is also, by far, the most common language.

#### Cast and crew

Size

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
ax1.hist(complete_df['cast_size'])
ax1.set_title('Cast size histogram')
ax2.hist(complete_df['crew_size'])
ax2.set_title('Crew size histogram')
plt.show()

Number of famous crew / cast members per movie

In [ ]:
complete_df['crew_top_50_counter'].value_counts()

In [ ]:
complete_df['cast_top_50_counter'].value_counts()

The list of movies with large number of famous cast includes Pulp Fiction, Megamind and Good Will Hunting

In [ ]:
complete_df.loc[complete_df['cast_top_50_counter'] >= 4]

#### Production countries

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
sns.boxplot(x='prod_countries_USA', y='ln_revenue', data=complete_df)
plt.title('Log-revenue for USA x non-USA prod. countries')
plt.ylabel("Log-revenue")
plt.xlabel("Prod. country == USA")
sns.despine() # Removes some of the borders

# Second plot
plt.subplot(1, 2, 2)
sns.boxplot(x='prod_countries_other_top_10', y='ln_revenue', data=complete_df)
plt.title('Log-revenue for other top 10 most common prod. countries')
plt.ylabel("Log-revenue")
plt.xlabel("Prod. country belongs to the other top 10 most frequent (USA incl.)?")
sns.despine()

In [ ]:
sns.boxplot(x='prod_countries_other_top_10', y='ln_revenue', data=complete_df.loc[complete_df['prod_countries_USA'] == 0])
plt.title('Revenue for other top 10 most common prod. countries')
plt.ylabel("Log-revenue")
plt.xlabel("Prod. country belongs to the other top 10 most frequent (USA excl.)?")
sns.despine()

Movies produced in the United States show greater revenues than their counterparts. Movies made on the other top 10 countries have almost similar performance to those made outside them. However, when excluding the 'made in USA' films from the comparison, the other top 10 countries perfom better than their counterparts.

#### Homepage

In [ ]:
sns.boxplot(x='has_homepage', y='ln_revenue', data=complete_df)
plt.title('Revenue comparison for movies with and without homepages')
plt.ylabel("Log-revenue")
plt.xlabel("Has homepage?")
plt.show()

As can be seen from the data, movies with homepages presents slightly higher revenues in comparison with those without website pages. However, the website itself probably is not a useful piece of information; therefore, this piece of information will be dropped later on.

#### Belongs to collection

In [ ]:
sns.boxplot(x='has_collection', y='ln_revenue', data=complete_df)
plt.ylabel("Log-revenue")
plt.xlabel("Movie belongs to collection?")
plt.show()

Movies that are part of franchises shows higher revenues.

## Dummies for date variables

In [ ]:
complete_df = pd.get_dummies(complete_df, columns=['release_quarter'], drop_first = True)
complete_df['release_weekday_friday'] = complete_df['release_weekday'].apply(lambda x: 1 if x == 4 else 0)

## Dropping useless variables

Since "original_lang_en" and "prod_countries_USA" are highly correlated, we'll drop the former.

In [ ]:
# Drops useless variables
df = complete_df.drop(['id','imdb_id', 'original_title', 'original_language', 'status', 'poster_path',
                  'belongs_to_collection', 'homepage', 'spoken_languages', 'tagline', 'overview',
                  'prod_countries', 'prod_companies', 'crew', 'cast', 'genres',
                  'belongs_to_collection', 'budget', 'Keywords', 'title', 'popularity','prod_countries_all',
                  'release_date', 'genres_count', 'original_lang_en', 'release_weekday', 'revenue'],
                 axis = 1) # Axis = 1 means you're dropping a COLUMN, not a row

In [ ]:
df.info()

In [ ]:
df.to_excel('df.xlsx')

### Correlation heatmap

In [ ]:
"""
df_without_release_date = df.drop(['release_weekday_2', 'release_weekday_2','release_weekday_3', 'release_weekday_4',
                                    'release_weekday_5', 'release_weekday_6', 'release_month_2', 'release_month_3', 
                                    'release_month_4', 'release_month_5', 'release_month_6', 'release_month_7', 
                                    'release_month_8', 'release_month_9', 'release_month_10', 'release_month_11', 
                                    'release_month_12'], axis = 1) 

corr = df_without_release_date.corr()
sns.heatmap(corr, annot = True, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
"""

# 3) Train-test split

**3)** Divida os dados em treino/teste: Sua base é balanceada? Você precisa pensar em alguma estratégia especial para dividir a sua base? Você considera estratégias como *K-fold validation*?

In [ ]:
# Models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

# Dimensionality reduction
from sklearn.preprocessing import StandardScaler # To use on X (features) before the PCA
from sklearn.decomposition import PCA

# Model selection
from sklearn import model_selection

# Performance metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_squared_log_error

In [ ]:
# Splitting the data stratifying by "prod country == USA"
df_train, df_test = model_selection.train_test_split(df, test_size = 0.3, random_state = 0, stratify = df['prod_countries_USA'])
print(df_train['prod_countries_USA'].mean())
print(df_test['prod_countries_USA'].mean())

# 4) Benchmark model

**4)** Construa o seu *benchmark*. Sempre comece com modelo bem simples que você tenha controle. Uma boa escolha é um modelo linear em que  você consiga interpretar os seus resultados/ter uma ideia das variáveis que podem influenciar os seus resultados.

### Model

In [ ]:
class MySklearningModel:
    """ Object for the regression models. Accepts the use of PCA beforehand, and also
    has a method for running cross-validation
    
    TO BE IMPLEMENTED:
    - Graphs for the cross-validation comparisons
    - Method for printing fitted coefficients
    """
    
    def __init__(self, model, df_train, df_test, independent_variable_list, dependent_variable, 
                 use_pca = False, pca_variance = 0.95):
        """ Initiates
        
        model: model's type
        df_train, df_test: train and test dataframes
        independent_variable_list: features (X variables)
        dependent_variable: target (y variable)
        use_pca: whether to use or not PCA for dimensionality reduction (default: false)
        pca_variance: explained variance threshold for the PCA. Goes from zero (0%) to one (100%) (default: 0.95)
        """
        self.model = model
        self.independent_variable_list = independent_variable_list
        self.dependent_variable = dependent_variable
        self.X_train, self.X_test = df_train[self.independent_variable_list].values, df_test[self.independent_variable_list].values
        self.y_train, self.y_test = np.squeeze(df_train[[self.dependent_variable]].values), np.squeeze(df_test[[self.dependent_variable]].values)
        self.pca_variance = pca_variance
        
        # Dimensionality Reduction
        if use_pca == True:
            # Rescaling before the PCA (necessary)
            my_scaler = StandardScaler()
            new_dataset = my_scaler.fit_transform(self.X_train) # Rescales
            self.X_test = my_scaler.transform(self.X_test) # Rescales

            pca = PCA(n_components = self.pca_variance)
            self.X_train = pca.fit_transform(new_dataset)
            self.X_test = pca.transform(self.X_test)
            
        return None

    
    def run_sklearn_regression_crossval(self, number_splits, score_list, random_state = 0):
        """ Runs the regression (USE IT FOR CROSS-VALIDATION)
        
        number_splits: number of splits for the kfold
        score_list: performance's metrics to be used
        random_state: random seed (default = 0)
        """
        kfold = model_selection.KFold(n_splits=number_splits, shuffle=True, random_state=random_state)
        results = model_selection.cross_validate(self.model, self.X_train, self.y_train, cv=kfold, scoring=score_list,return_train_score=True)
        
        print(str(self.model))
        
        for score in score_list:
            print(score+':')
            print('Cross-val Train: '+'Mean',np.mean(results['train_'+score]),
            'Standard Error',np.std(results['train_'+score]))
            print('Cross-val Test: '+'Mean',np.mean(results['test_'+score]),
            'Standard Error',np.std(results['test_'+score]))
            print("")
            
    def run_sklearn_regression(self, score_list):
        """ Runs the regression """
        # Fits the model and predicts y
        self.model.fit(self.X_train, self.y_train)
        self.y_pred = self.model.predict(self.X_test)
        
        print(self.model)
    
        # Prints the performance metrics:
        for score in score_list:
            # If opted for MSE, returns RMSE
            if score == "mean_squared_error":
                print("root_" + score + ": " + str(eval(score + '(self.y_test, self.y_pred, squared = False)')))
            else:
                print(score + ": " + str(eval(score + '(self.y_test, self.y_pred)')))
        
        return None
    
    def prints_parameters(self):
        """ [NOT WORKING PROPERLY] Prints the model's parameters"""
        # Checks whether the model has already been fitted or not.
        if hasattr(self.model, 'coef_'):
            for i in range(len(self.independent_variable_list)):
                print(self.independent_variable_list[i] + ": " + str(self.model.coef_[i]))
        else:
            print("Model has not yet been fitted.")
        
        return None  

In [ ]:
if __name__ == '__main__':
    
    number_splits=5
    
    dependent_variable = 'ln_revenue'
    independent_variable_list = df.columns.values
    independent_variable_list = [x for x in independent_variable_list if x != dependent_variable]
    
    for model in [LinearRegression(), KNeighborsRegressor(n_neighbors = 10),
                  RandomForestRegressor(n_estimators = 100, max_depth = 10, random_state = 0)]:
        
        my_model = MySklearningModel(model,df_train, df_test, independent_variable_list,dependent_variable, 
                                     use_pca = True, pca_variance = 0.99)
        
        # Cross-validation
        my_model.run_sklearn_regression_crossval(number_splits, ['neg_mean_squared_error', 'r2'])
        
        # Regression
        #my_model.run_sklearn_regression(['mean_squared_error', 'r2_score'])
        
        print("\n\n")

****

# CHECK IF BELOW CODE IS __REALLY__ RELEVANT

The following code was used for cross-validation

In [ ]:
if __name__ == '__main__':
    
    number_splits=5
    
    dependent_variable = 'ln_revenue'
    independent_variable_list = df.columns.values
    independent_variable_list = [x for x in independent_variable_list if x != dependent_variable]
    
    for i in range(1, 15):
        print("Number of neighbours: ", i)
        
        model = KNeighborsRegressor(n_neighbors = i)
        my_model = MySklearningModel(model,df_train, df_test,
        independent_variable_list,dependent_variable)
        
        my_model.run_sklearn_regression_crossval(number_splits,
        ['neg_mean_squared_error', 'r2'])
        #my_model.run_sklearn_regression(['mean_squared_error', 'r2_score'])
        #print("\nParameters:")
        #my_model.prints_parameters()
        print("\n\n")

Using cross-validation, I opted to run the KNeighborsRegressor using n = 10 neighbors. After this threshold, the gains on the tests' scores were marginal.

# **6)** Dimensionality Reduction

**6)** Você precisará de um procedimento específico de seleção de variáveis?

## Principal Component Analysis (PCA)

In [ ]:
if __name__ == '__main__':
    
    dependent_variable = 'ln_revenue'
        
    independent_variable_list = df.columns.values
    independent_variable_list = [x for x in independent_variable_list if x != dependent_variable]
    
    scaler = MinMaxScaler()
    
    X = df.drop([dependent_variable], axis = 1) # Features
    #X = df[independent_variable_list].values
    X = scaler.fit_transform(X)
    
    print(X)
    y = df[dependent_variable] # Target variable

    
    
    
    pca = PCA()
    pca.fit(X)
    cumsum = np.cumsum(pca.explained_variance_ratio_)
    d = np.argmax(cumsum >= 0.95) + 1
    print(d)

    # Another way around:        
    pca = PCA(n_components=0.95)
    projected2again = pca.fit_transform(X)
    
    # Yet another way:
    plt.figure(0)
    plt.plot(cumsum)

In [ ]:
print(projected2again)

In [ ]:
if __name__ == '__main__':

    dependent_variable = 'ln_revenue'
    
    X = df.drop([dependent_variable], axis = 1) # Features
    y = df[dependent_variable] # Target variable
    
    independent_variable_list = df.columns.values
    independent_variable_list = [x for x in independent_variable_list if x != dependent_variable]
    
    """
    pca = PCA(1)
    pca.fit(X)
    projected2again = pca.fit_transform(X)    

    my_projection_X = []
    my_projection_y = []

    for i in range(len(projected2again)):
        my_projection_X.append(projected2again[i][0])

    my_projection_y = [value for value in y.values]

    d = {'y':my_projection_y,'X':my_projection_X}
    df_post_pca = pd.DataFrame(d)
    """
    
    n_components = 15
    pca = PCA(n_components)
    df_post_pca = pd.DataFrame(pca.fit_transform(X), columns=['PCA%i' % i for i in range(n_components)], index=X.index)
    df_post_pca['y'] = df[dependent_variable]

    
    
    number_splits=5
    
    dependent_variable = 'y'
    independent_variable_list = df_post_pca.columns.values
    independent_variable_list = [x for x in independent_variable_list if x != dependent_variable]
    
    for model in [LinearRegression(), KNeighborsRegressor(n_neighbors = 10),
                  RandomForestRegressor(n_estimators = 100, max_depth = 10, random_state = 0)]:
        my_model = MySklearningModel(model,df_post_pca,
        independent_variable_list,dependent_variable)
        
        print(str(model) + ":")
        my_model.run_sklearn_regression_crossval(number_splits,
        ['neg_mean_squared_error', 'r2'])
        print("\n\n")